<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/final_model_error_analysis_LP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Error analysis of final model



In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

In [2]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [0]:
# Load the test data and predictions
# dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)
# test_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/test.csv", index_col=0)
# dev_preds = pd.read_csv("gs://fb-congressional-data/models/dev_preds.csv")
# test_preds = pd.read_csv("gs://fb-congressional-data/models/test_preds.csv")
dev_preds_party = pd.read_csv("gs://fb-congressional-data/models/dev_preds_party.csv")
test_preds_party = pd.read_csv("gs://fb-congressional-data/models/test_preds_party.csv")


In [6]:
test_preds.head()

,original_idx,probs,preds
0,9225551,0.398534,M
1,7514285,0.363646,M
2,6128273,0.283775,M
3,6370382,0.320124,M
4,8962461,0.316799,M


In [45]:
test_preds_party.head()

,original_idx,probs_party,preds_party
0,9225551,0.384233,Congress_Republican
1,7514285,0.545365,Congress_Democrat
2,6128273,0.400835,Congress_Republican
3,6370382,0.327347,Congress_Republican
4,8962461,0.461587,Congress_Republican


In [12]:
dev_preds.head()

,original_idx,probs,preds
0,2210805,0.288363,M
1,10316428,0.253759,M
2,5297343,0.252464,M
3,5030266,0.293893,M
4,4918178,0.176300,M


In [46]:
dev_preds_party.head()


,original_idx,probs_party,preds_party
0,2210805,0.653872,Congress_Democrat
1,10316428,0.457214,Congress_Republican
2,5297343,0.353170,Congress_Republican
3,5030266,0.544343,Congress_Democrat
4,4918178,0.144020,Congress_Republican


In [47]:
test_df = test_df.merge(test_preds,on='original_idx')
test_df = test_df.merge(test_preds_party,on='original_idx')
test_df.head()

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds,probs_party,preds_party
0,1682766,78379937,W,21367,Tonia,jacky thank you for addressing this very troubling news! please demand a full independent invest...,Jacky Rosen,Congress_Democratic,0.286659,M,0.567257,Congress_Democrat
1,1682767,78379937,W,21367,Michancy,agreed! the russian connection to this administration must be fully investigated. flynn is the ...,Jacky Rosen,Congress_Democratic,0.150482,M,0.439200,Congress_Republican
2,1682768,78379937,W,21367,Andrea,please push chaffez and cummings to look into this!,Jacky Rosen,Congress_Democratic,0.203123,M,0.508537,Congress_Democrat
3,1682769,78379937,W,21367,Tina,i could not agree more!,Jacky Rosen,Congress_Democratic,0.229004,M,0.370849,Congress_Republican
4,1682770,78379937,W,21368,Cathy,"i imagine that is followed by ""when qualifited.""",Jacky Rosen,Congress_Democratic,0.275994,M,0.354149,Congress_Republican


In [48]:
# dev_df = dev_df.merge(dev_preds,on='original_idx')
dev_df = dev_df.merge(dev_preds_party,on='original_idx')
dev_df.head()

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds,probs_party,preds_party
0,1965080,86681682,M,29909,James,thanks for passing this ridiculous legislation. https://www.youtube.com/watch?v=-y4wd8royre,Jim Banks,Congress_Republican,0.208421,M,0.391387,Congress_Republican
1,1965081,86681682,M,29909,Martha,handsome young man on the left. 😂 mom,Jim Banks,Congress_Republican,0.257950,M,0.436190,Congress_Republican
2,1965082,86681682,M,29911,Preston,you are sideways!,Jim Banks,Congress_Republican,0.364659,M,0.484376,Congress_Republican
3,1965083,86681682,M,29911,Bobby,hi congressman banks,Jim Banks,Congress_Republican,0.035806,M,0.493345,Congress_Republican
4,1965084,86681682,M,29911,Landon,"jim, i love your office. the 5th floor is for winners.",Jim Banks,Congress_Republican,0.048077,M,0.546275,Congress_Democrat


In [0]:
wrong_preds = test_df[test_df.op_gender!=test_df.preds]


In [18]:
wrong_preds.op_gender.value_counts()

W    219299
M     57502
Name: op_gender, dtype: int64

In [50]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.03995132955674003

In [51]:
#random sample of mistakes
wrong_preds.sample(10)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds,probs_party,preds_party
72927,2334550,98470558,W,50405,Stephen,what about balancing the budget? if we are going behind because of the aca or other our of cont...,Elise Stefanik,Congress_Republican,0.203921,M,0.175891,Congress_Republican
1391124,13695667,83604908,W,534889,B.,win baby!,Shelley Moore Capito,Congress_Republican,0.332606,M,0.323022,Congress_Republican
1384380,13688555,83604908,W,534736,Diane,shelley i am torn apart with our pres. having thousands of foreign soljors here from the un. and...,Shelley Moore Capito,Congress_Republican,0.189369,M,0.318477,Congress_Republican
446420,6344060,67504140,M,233336,Keith,"kay hagan has been nothing more than obama's rubber stamp since 2009. now, she expects people to...",Thom Tillis,Congress_Republican,0.548123,W,0.122853,Congress_Republican
705432,8911098,3162230,W,305345,Tim,just my luck. streaming cut out and i missed all but the last 2 minutes but from what i did hear...,Kristi Noem,Congress_Republican,0.266814,M,0.312440,Congress_Republican
313436,4126921,72264613,W,147923,Efrem,"if i were to give that answer, i would say yes (channeling winston zeddmore from ghostbusters) t...",Kyrsten Sinema,Congress_Democratic,0.227072,M,0.311274,Congress_Republican
306694,4119675,72264613,W,147644,Tim,when/where is your next book signing?,Kyrsten Sinema,Congress_Democratic,0.298402,M,0.438716,Congress_Republican
632984,7606969,47811014,W,292574,Jaye,welcome back. I am glad you got the help you needed hopefully more will get back to their families.,Chellie Pingree,Congress_Democratic,0.345052,M,0.502120,Congress_Democrat
341861,5801475,62031614,W,191604,Randy,repeal times 17 trillion,Jackie Walorski,Congress_Republican,0.186514,M,0.147731,Congress_Republican
273379,3501547,33093506,W,114380,Dianne,http://indiancountrytodaymedianetwork.com/2016/11/27/after-month-conflict-mass-media-arrives-for...,Tammy Baldwin,Congress_Democratic,0.298474,M,0.446861,Congress_Republican


In [52]:
# most confident wrong predictions where true was M but prediction was W
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds,probs_party,preds_party
362434,6096572,39064513,M,209578,Sue,"thank you, bernie. thank you, jill. thank you, tulsi. thank you to all the veterans. and a s...",Earl Blumenauer,Congress_Democratic,0.999994,W,0.999999,Congress_Democrat
933902,9321039,61839194,M,327804,Kenneth,"hey kamala, i voted for you because of the great work you have done for the environmental justic...",Cory Booker,Congress_Democratic,0.999930,W,0.999951,Congress_Democrat
933563,9320679,61839194,M,327804,Judy,go kamala. win!,Cory Booker,Congress_Democratic,0.999873,W,0.999685,Congress_Democrat
933715,9320844,61839194,M,327804,Kristina,i love and appreciate all three of you! kamala has all five of our votes!!👍,Cory Booker,Congress_Democratic,0.999848,W,0.999400,Congress_Democrat
381625,6116832,39064513,M,210160,Claudia,what is tulsi doing in portland ? she is hawaii's congresswoman. doing a wonderful job.,Earl Blumenauer,Congress_Democratic,0.999845,W,0.999877,Congress_Democrat
860437,9240975,61839194,M,327395,Cindy,thank you patty murray. i knew i voted for you for a good reason.,Cory Booker,Congress_Democratic,0.999827,W,0.999934,Congress_Democrat
933860,9320995,61839194,M,327804,Lynn,kamala will be a most wonderful addition to the us senate..heart thanks for hair support...💌,Cory Booker,Congress_Democratic,0.999814,W,0.999936,Congress_Democrat
926925,9313361,61839194,M,327789,Ralph,"good job corey, better job patty for keeping your hand on the pulse of america.",Cory Booker,Congress_Democratic,0.999745,W,0.994470,Congress_Democrat
857375,9237795,61839194,M,327395,Ann,i <3 patty murray. and you. thank you. i <3 wa state.,Cory Booker,Congress_Democratic,0.999650,W,0.999497,Congress_Democrat
933573,9320689,61839194,M,327804,John,"stay close to them, kamala--that has the future.",Cory Booker,Congress_Democratic,0.999585,W,0.999484,Congress_Democrat


In [53]:
# most confident wrong predictions where true was W but predicted M
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds,probs_party,preds_party
633199,7607197,47811014,W,292581,Mark,if angus votes for him angus will lose my vote next cycle.,Chellie Pingree,Congress_Democratic,0.000052,M,0.196593,Congress_Republican
667404,8871902,3162230,W,303890,Joelle,vote gohmert! and remember you are there for america not just sd. very disappointed you voted fo...,Kristi Noem,Congress_Republican,0.000072,M,0.000129,Congress_Republican
75588,2337351,98470558,W,50839,Katie,go louie :),Elise Stefanik,Congress_Republican,0.000078,M,0.000933,Congress_Republican
681997,8887071,3162230,W,304367,Carl,it would seem you are an old hand at dealing with clowns like that. keep up the good fight sir!,Kristi Noem,Congress_Republican,0.000106,M,0.162795,Congress_Republican
638638,7612806,47811014,W,292642,Kathy,we must be sure to let rep. poliquin know that we are watching how he votes on this.,Chellie Pingree,Congress_Democratic,0.000146,M,0.000494,Congress_Republican
638455,7612619,47811014,W,292641,Ren,"so, just wondering is rep poliquin going to vote against obama care and strip 75,000 mainers fr...",Chellie Pingree,Congress_Democratic,0.000174,M,0.005336,Congress_Republican
353324,5928285,15083367,W,201408,Heather,"you do realize you only won by 5.8 %. start doing your job, show up and listen to your constitue...",Barbara Comstock,Congress_Republican,0.000281,M,0.000075,Congress_Republican
642983,7617345,47811014,W,292986,Sam,"sending good thoughts, angus!",Chellie Pingree,Congress_Democratic,0.000307,M,0.106553,Congress_Republican
289660,4101545,72264613,W,146140,Don,"what a phony you are! I am so proud of my rep, ruben gallego.",Kyrsten Sinema,Congress_Democratic,0.000354,M,0.998451,Congress_Democrat
1391273,13695821,83604908,W,534896,Susie,"tennant out this time, carper out the next time. shelley has my vote. jim ballard",Shelley Moore Capito,Congress_Republican,0.000362,M,0.961506,Congress_Democrat


In [0]:
# Least confident predictions.

least_conf = test_df[(test_df.probs<0.50005) & (test_df.probs>0.49995)]

In [55]:
# Incorrect low confidence predictions.
least_conf[least_conf.op_gender!=least_conf.preds].sort_values(by='probs')


,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds,probs_party,preds_party
339155,5797921,62031614,W,191300,Jen,seriously? unemployment is at 5% and the stock market is still at a all time high .... actions a...,Jackie Walorski,Congress_Republican,0.499951,M,0.431080,Congress_Republican
296202,4108647,72264613,W,146797,Michael,it must be an election year. the prada socialist ( her words) claiming to be for the good sheepl...,Kyrsten Sinema,Congress_Democratic,0.499955,M,0.255204,Congress_Republican
290034,4101964,72264613,W,146192,William,"omg, I am agreeing with ms. sinema.",Kyrsten Sinema,Congress_Democratic,0.499982,M,0.652048,Congress_Democrat
860607,9241152,61839194,M,327395,Ann,some are so hung up on the abortion issue that they ignore all the other issues of women's health,Cory Booker,Congress_Democratic,0.500004,W,0.514635,Congress_Democrat
1413104,13822951,8879946,M,540664,Kristen,what if the breast cancer patient is on medicaid?,Bill Cassidy,Congress_Republican,0.500011,W,0.554036,Congress_Democrat
423770,6320227,67504140,M,233105,Kesha,hagan knows we all now how she misused that stimulus for her personal use! crime,Thom Tillis,Congress_Republican,0.500014,W,0.354412,Congress_Republican
214771,3098496,42138413,M,81676,Loretta,we need more senators like you. wish you were here in florida,Tim Scott,Congress_Republican,0.500020,W,0.579296,Congress_Democrat
975392,10019419,41098373,M,364679,Kerry,i am proud that my congressional representative dwight evans joined john lewis and others in boy...,Dwight Evans,Congress_Democratic,0.500020,W,0.992244,Congress_Democrat
984541,11114055,10284504,M,391015,Carol,proud of our west salem girl!,Ron Kind,Congress_Democratic,0.500041,W,0.464439,Congress_Republican
1162129,12270735,18759563,M,458809,Milton,"actually, tom cotton did not violate the logan act which applies to private citizens negotiating...",Tom Cotton,Congress_Republican,0.500043,W,0.624831,Congress_Democrat


In [56]:
# Correct low confidence predictions.
least_conf[least_conf.op_gender==least_conf.preds].sort_values(by='probs')


,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds,probs_party,preds_party
165660,3046932,42138413,M,80946,Brigitte,"happy mother's day to your mother, a strong woman!",Tim Scott,Congress_Republican,0.499951,M,0.648575,Congress_Democrat
784959,9160751,61839194,M,327345,Nicky,"as the parent of an amazing daughter with special needs, i want to thank you from the bottom of ...",Cory Booker,Congress_Democratic,0.499951,M,0.586767,Congress_Democrat
360042,6094104,39064513,M,209554,Jeannette,"thank you for your words about the hero john lewis. and, for your stand in boycotting the inaugu...",Earl Blumenauer,Congress_Democratic,0.499961,M,0.996039,Congress_Democrat
774333,9149525,61839194,M,327340,Giselle,"thank you for calling this out, senator booker! she had the right to read this letter!",Cory Booker,Congress_Democratic,0.499963,M,0.598405,Congress_Democrat
783367,9159119,61839194,M,327342,Betty,yeah well i am still deeply upset that you sold out to pharma...,Cory Booker,Congress_Democratic,0.499965,M,0.744950,Congress_Democrat
182677,3064882,42138413,M,81170,Betty,what fine men their mother did something great when she raised them.,Tim Scott,Congress_Republican,0.499965,M,0.385799,Congress_Republican
543695,7484818,65473098,M,288121,Jonn,"we tried and look what they did to us in michigan to keep it off the ballot,sorry but we should ...",Thomas Massie,Congress_Republican,0.499984,M,0.566431,Congress_Democrat
431961,6328802,67504140,M,233157,BillandKay,yes let us move her out,Thom Tillis,Congress_Republican,0.499986,M,0.324856,Congress_Republican
421992,6318375,67504140,M,233095,John,what would she do if thom tillis walked in? run and hide again?,Thom Tillis,Congress_Republican,0.499987,M,0.301572,Congress_Republican
104725,2981803,42138413,M,80121,Roseanne,"i am a south carolinian and a retired professor, educated through the public school system. bets...",Tim Scott,Congress_Republican,0.499988,M,0.290688,Congress_Republican
